In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

 According to [PDG, Rev. Resonances](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15) the once-substracted dispersion integral is given by:

# $T$ matrix definition

In [ ]:
class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):
        return printer._print(self.args[0])


n = 2
I = sp.Identity(n)
K = sp.MatrixSymbol("K", n, n)
CM = DiagonalMatrix(sp.MatrixSymbol(R"\Sigma^l", n, n))
FF = DiagonalMatrix(sp.MatrixSymbol(R"n", n, n))
Math(aslatex({CM: CM.as_explicit()}))

According to [PDG, Rev. Resonances](https://pdg.lbl.gov/2024/reviews/rpp2024-rev-resonances.pdf) the scattering amplitude is given by:

$$
M = n \left( I - i K \rho n^2 \right)^{-1} K n
$$

where $n$ is the barrier factor matrix and $\rho $ the phase space factor matrix. Both are diagonal matrices containing the respective factors for each channel. 
The factor $i\rho n^2$ is replaced by the once-subtracted Chew--Mandelstam dispersion integral $\Sigma^l$.

In [ ]:
T_I = FF * (I + K * CM).inv() * K * FF
T_I

In [ ]:
T_I_explicit = T_I.as_explicit()
T_I_explicit[0, 0].simplify(doit=False)

In [ ]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
g1 = sp.Symbol(R"g^{0}_1")
g2 = sp.Symbol(R"g^{0}_2")

In [ ]:
parameters = {
    ma1: 1.0,
    mb1: 1.5,
    ma2: 1.5,
    mb2: 2.0,
    m0: 3.0,
    g1: 0.7,
    g2: 0.7,
}

## Symbolic dispersion integral 

In [ ]:
s, m1, m2, L = sp.symbols("s, m1, m2, L", nonnegative=True)
L_val = 1

In [ ]:
integral_expr = ChewMandelstamIntegral(s, m1, m2, L)
integral_expr.doit(deep=False)

In [ ]:
integral_p_wave_func = sp.lambdify(
    [s, m1, m2, integral_expr.epsilon],
    integral_expr.subs(L, L_val).doit(),
)

## $T$ matrix parametrization

In [ ]:
k_expr_00 = (g1 * g1 * m0) / (s - m0**2)
k_expr_10 = (g1 * g2 * m0) / (s - m0**2)
k_expr_11 = (g2 * g2 * m0) / (s - m0**2)

T_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    FF[0, 0]: FormFactor(s, ma1, mb1, angular_momentum=L_val),
    FF[1, 1]: FormFactor(s, ma2, mb2, angular_momentum=L_val),
}

Math(aslatex(T_expressions))

In [ ]:
T_I_cm_expr = T_I_explicit.xreplace(T_expressions)
T_I_cm_expr[0, 0].simplify(doit=False)

## Sheets II, III, and IV

In the case of two channels, there are four Riemann sheets. The first sheet ([Sheet I](#riemann-sheet-i)) is physical and three unphysical ones. The physical sheet is calculated using the analytic solution of the Chew-Mandelstam function.
The other sheets are reached by adding the discontinuity across the branch cut:
For higher $l$:
$$
\operatorname{Disc} \varSigma_\ell(s) \;=\; 2i\,\rho(s)\,n_\ell^2(s)
$$
E.g.:

$$
T^{II}=n\left(\mathbb{I} - K (\varSigma_\ell + 2 i \rho)\right)^{-1} K n
$$

Depending on the centre-of-mass energy, different Riemann sheets connect smoothly to the physical one. Therefore, two cases are studied: one where the resonance mass is above the threshold of the second and first channel, and another where the resonance mass is between the threshold of the first and second channel. For the 2 channel one gets:

$$
\begin{eqnarray}
\operatorname{Disc}_{\mathrm{I,II}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}\rho_1 n_{\ell,1}^2(s) & 0  \\ 0 & 0 \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,III}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}\rho_1 n_{\ell,1}^2(s) & 0  \\ 0 & \rho_2 n_{\ell,2}^2(s) \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,IV}} \varSigma_\ell(s)
&=& 2 i\left[\begin{array}{rr}0 & 0  \\ 0& \rho_2  n_{\ell,2}^2(s) \end{array}\right].
\end{eqnarray}
$$

In [ ]:
rho = DiagonalMatrix(sp.MatrixSymbol("rho", n, n))
Math(aslatex({rho: rho.as_explicit()}))

In [ ]:
T_II = (T_I.inv() + 2 * sp.I * rho * FF**2).inv()
T_III = (T_I.inv() + 2 * sp.I * rho * FF**2).inv()
T_IV = (-T_I.inv() - 2 * sp.I * rho * FF**2).inv()
Math(aslatex([T_II, T_III, T_IV]))

In [ ]:
T_II_explicit = T_II.as_explicit()
T_II_explicit[0, 0].simplify(doit=False)

In [ ]:
T_III_explicit = T_III.as_explicit()
T_III_explicit[0, 0].simplify(doit=False)

In [ ]:
T_IV_explicit = T_IV.as_explicit()
T_IV_explicit[0, 0].simplify(doit=False)

In [ ]:
T_expressions_II = {
    **T_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: 0,
}
T_expressions_III = {
    **T_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}
T_expressions_IV = {
    **T_expressions,
    rho[0, 0]: 0,
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}

In [ ]:
T_II_rho_expr = T_II_explicit.xreplace(T_expressions_II)
T_III_rho_expr = T_III_explicit.xreplace(T_expressions_III)
T_IV_rho_expr = T_IV_explicit.xreplace(T_expressions_IV)

# Plot

In [ ]:
def evaluate(func, s_array):
    args = transform(s_array)
    return func(*args)


def transform(s_array):
    cm1_array = integral_p_wave_func(
        s_array, parameters[ma1], parameters[mb1], epsilon=1e-3
    )
    cm2_array = integral_p_wave_func(
        s_array, parameters[ma2], parameters[mb2], epsilon=1e-3
    )
    return s_array, cm1_array, cm2_array

In [ ]:
symbols = (s, CM[0, 0], CM[1, 1])
T_I_func = sp.lambdify(symbols, T_I_cm_expr[0, 0].subs(parameters).doit())
T_II_func = sp.lambdify(symbols, T_II_rho_expr[0, 0].subs(parameters).doit())
T_III_func = sp.lambdify(symbols, T_III_rho_expr[0, 0].subs(parameters).doit())
T_IV_func = sp.lambdify(symbols, T_IV_rho_expr[0, 0].subs(parameters).doit())

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 8, num=300)
y = np.linspace(epsilon, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

In [ ]:
T1n_res1 = evaluate(T_I_func, Zn**2)
T1p_res1 = evaluate(T_I_func, Zp**2)

T2n_res1 = evaluate(T_II_func, Zn**2)
T2p_res1 = evaluate(T_II_func, Zp**2)

T3n_res1 = evaluate(T_III_func, Zn**2)
T3p_res1 = evaluate(T_III_func, Zp**2)

T4n_res1 = evaluate(T_IV_func, Zn**2)
T4p_res1 = evaluate(T_IV_func, Zp**2)

In [ ]:
%config InlineBackend.figure_formats = ["png"]

fig, axes = plt.subplots(figsize=(12, 8), ncols=2, nrows=2, sharey=True)
ax1, ax2, ax3, ax4 = axes.flatten()

fig.delaxes(ax3)
fig.delaxes(ax4)

for ax in [ax1, ax2]:
    ax.set_xlabel(R"$\mathrm{Re}\,\sqrt{s}$")
ax1.set_ylabel(R"$\mathrm{Im}\,\sqrt{s}$")

ax1.set_title("I and II")
ax2.set_title("I and III")


T_max = 2
style = dict(vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)

mesh = ax1.pcolormesh(X, Y, T1p_res1.imag, **style)
ax1.pcolormesh(X, -Y, T2n_res1.imag, **style)

ax2.pcolormesh(X, Y, T1p_res1.imag, **style)
ax2.pcolormesh(X, -Y, T3n_res1.imag, **style)


s_thr1 = parameters[ma1] + parameters[mb1]
s_thr2 = parameters[ma2] + parameters[mb2]
linestyle = dict(ls="dotted", lw=1)

for ax in [ax1, ax2]:
    ax.axhline(0, c="black", **linestyle)
    ax.axvline(s_thr1, c="C0", **linestyle, label=R"$\sqrt{s_\mathrm{thr1}}$")
    ax.axvline(s_thr2, c="C1", **linestyle, label=R"$\sqrt{s_\mathrm{thr2}}$")

linestyle_res = dict(c="r", ls="dotted", label=R"$m_\mathrm{res}$")
for ax in [ax1, ax2]:
    ax.axvline(parameters[m0], **linestyle_res)

ax2.legend()

fig.text(0.5, 0.93, R"$s_{thr1}<s_{thr2}<m_{res}$", ha="center", fontsize=18)
fig.text(0.5, 0.46, R"$s_{thr1}<m_{res}<s_{thr2}$", ha="center", fontsize=18)

fig.subplots_adjust(wspace=1)

cax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(mesh, cax=cax)
cbar.ax.set_title(R"$\mathrm{Im} T(s)$")

fig.tight_layout(rect=[0, 0.03, 0.9, 0.95])
fig.show()